In [26]:
import pandas as pd
import numpy as np
import preprocessor as p
import emoji
import emot
import  re


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
df = pd.read_csv('../collection/data/tweet_rescrape.csv')
bot_accounts = pd.read_csv('data/bot_id_username.csv')
clean_accounts = pd.read_csv('data/non_bot_accounts.csv')

bot_tweets = df[df['username'].isin(bot_accounts['username'])].reset_index(drop=True)

clean_tweets = df[df['username'].isin(clean_accounts['username'])]

bot_tweets['BotOrNot'] = 1
clean_tweets['BotOrNot'] = 0

combined_df = bot_tweets.append(clean_tweets, ignore_index=True)

new_df = combined_df.sample(frac=1).reset_index(drop=True)

/home/warproxxx/.local/lib/python3.5/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [28]:
def tokenize_tweet(tweet):    
    for word in tweet.split(' '):
        if (word.isalnum() and word == word.upper() and word.isdigit() == False and len(word) > 1):
            tweet = tweet.replace(word, word + " <allcaps>")
            
            
    tweet = emoji.demojize(tweet, delimiters=('<', '>'))

    p.set_options(p.OPT.HASHTAG, p.OPT.MENTION, p.OPT.NUMBER, p.OPT.RESERVED,p.OPT.URL)
    
    tokenized = p.tokenize(tweet)
    
    res = emot.emoticons(tokenized)
    
    if 'mean' in res:
        for i in range(len(res['mean'])):
            tokenized = tokenized.replace(res['value'][i], "<" + res['mean'][i].split(' ')[0] + ">")
    
    tokenized = tokenized.lower()
    tokenized = tokenized.replace('><', '> <')
    tokenized = tokenized.translate(str.maketrans('','',"!\"“'&(”)*+,’-./:;=?[\\]^`{|}~"))
    
    return tokenized

In [29]:
new_df['tweettext'] = new_df['tweettext'].apply(lambda x: tokenize_tweet(x))

In [30]:
new_df = new_df[['tweettext', 'retweets', 'username', 'timestamp', 'likes', 'replies', 'BotOrNot']]

In [31]:
new_df['hashtag_count'] = new_df['tweettext'].apply(lambda x: x.count("<hashtag>"))

In [32]:
new_df['mention_count'] = new_df['tweettext'].apply(lambda x: x.count("<mention>"))

In [33]:
new_df['url_count'] = new_df['tweettext'].apply(lambda x: x.count("<url>"))

In [34]:
new_df.to_csv('data/new_labeled_tweets.csv', index=False)

In [ ]:
# df['hashtags'] = df['hashtags'].apply(lambda x: convert_to_list(x))
# df['urls'] = df['urls'].apply(lambda x: convert_to_list(x))

# df['hastag_count'] = df['hashtags'].apply(lambda x: len(x))
# df['url_count'] = df['urls'].apply(lambda x: len(x))

# df = df[['tweet_text', 'retweet_count', 'like_count', 'reply_count', 'hastag_count', 'url_count']]

# df['mention_count'] = df['tweet_text'].str.count('@')